# 03. Data Complementaria al Proyecto
### Breve descripción del notebook
En este notebook vas a encontrar todo el desarrollo de la carga y manipulación de los dataset obtenidos de la página de Estadísticas y Censos de la Cuidad de Buenos aires.<br>
El propósito de es notebook es limpiar y corregir y transformar la información disponible para ser luego utilizada en el Dashboard del proyecto para el cálculo de KPIS y métricas que nos permitan usar esa información en el análisis y desarrollo del proyecto 
<br>
Comenzamos:

# 3.1. Cargamos las librerías necesarias

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

%load_ext autoreload
%autoreload 2
import funciones

import warnings
warnings.filterwarnings("ignore")


# 3.2 Cargamos los datasets disponibles a varios dataframes de pandas para luego limpiarlos, y combinarlos para obtener las tablas que nos serán útiles.

In [3]:
# Cargamos el dataset con sus diferentes páginas a pandas
sheets_to_load = ["2025", "2024","2023", "2022","2021", "2020","2019", "2018","2017", "2016"]
# Cargamos las diferentes hojas del excel a un diccionario 
sheets_data = pd.read_excel(r'data\PBP_CO_T_1025.xls ', sheet_name=sheets_to_load)

# Accedemos individualmente a los DataFrames usando el nombre de la pagina como fuente
d_25 = sheets_data["2025"]
d_24 = sheets_data["2024"]
d_23 = sheets_data["2023"]
d_22 = sheets_data["2022"]
d_21 = sheets_data["2021"]
d_20 = sheets_data["2020"]
d_19 = sheets_data["2019"]
d_18 = sheets_data["2018"]
d_17 = sheets_data["2017"]
d_16 = sheets_data["2016"]


In [4]:
d_25.head()

,Año,Edad,Total,Varones,Mujeres
0,2025,0,38465.3904,19809.808,18655.5824
1,2025,1,38729.7200,19942.096,18787.6240
2,2025,2,38931.4240,20042.472,18888.9520
3,2025,3,39077.5040,20114.376,18963.1280
4,2025,4,39174.9616,20161.248,19013.7136


In [5]:
# Voy a uir todos los  DataFrames en uno solo llamado poblacion_total
ds = [d_21, d_22, d_23, d_24, d_25, d_16, d_17,d_18,d_19, d_20]

# Concatenar todos los DataFrames 
poblacion_total = pd.concat(ds, ignore_index=True)

poblacion_total

,Año,Edad,Total,Varones,Mujeres
0,2021,0,40225.4512,20736.6400,19488.8112
1,2021,1,40404.0720,20808.0480,19596.0240
2,2021,2,40515.3360,20851.0160,19664.3200
3,2021,3,40566.4640,20868.1680,19698.2960
4,2021,4,40564.6768,20862.1280,19702.5488
...,...,...,...,...,...
705,2020,66,28358.2384,12014.9056,16343.3328
706,2020,67,27675.8784,11622.7536,16053.1248
707,2020,68,27086.8064,11270.2736,15816.5328
708,2020,69,26537.7376,10935.6048,15602.1328


In [6]:
poblacion_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Año      710 non-null    int64  
 1   Edad     710 non-null    object 
 2   Total    710 non-null    float64
 3   Varones  710 non-null    float64
 4   Mujeres  710 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 27.9+ KB


In [7]:
# Reemplazamos  '70 y más' en edad por 70 para que nos quede como int
poblacion_total['Edad'] = poblacion_total['Edad'].replace('70 y más', 70)
# Convertir todas las columnas a  integer datatype
poblacion_total = poblacion_total.astype(int)
poblacion_total

,Año,Edad,Total,Varones,Mujeres
0,2021,0,40225,20736,19488
1,2021,1,40404,20808,19596
2,2021,2,40515,20851,19664
3,2021,3,40566,20868,19698
4,2021,4,40564,20862,19702
...,...,...,...,...,...
705,2020,66,28358,12014,16343
706,2020,67,27675,11622,16053
707,2020,68,27086,11270,15816
708,2020,69,26537,10935,15602


In [8]:
poblacion_total.columns

Index(['Año', 'Edad', 'Total', 'Varones', 'Mujeres'], dtype='object')

## Vamos a transformar el dataframe para que nos sea util cuando lo importemos a Power BI

In [9]:
# Separamos el dtaframe en dos DataFrame por genero
male_df = poblacion_total[['Año', 'Edad', 'Total', 'Varones']].copy()
female_df = poblacion_total[['Año', 'Edad', 'Total', 'Mujeres']].copy()

# Agregamos a cada uno una columna genero
male_df['genero'] = 'varones'
female_df['genero'] = 'mujeres'

# Renombramos las columns
male_df.rename(columns={'Varones': 'Total_Parcial'}, inplace=True)
female_df.rename(columns={'Mujeres': 'Total_Parcial'}, inplace=True)

# Unimos de nuevo ambos DataFrames
poblacion_total = pd.concat([male_df, female_df])

# Hacemos un Sort del DataFrame por 'Año'
poblacion_total.sort_values(by='Año', inplace=True)

poblacion_total.reset_index(drop=True, inplace=True)
poblacion_total

,Año,Edad,Total,Total_Parcial,genero
0,2016,17,36705,18224,mujeres
1,2016,31,47487,24349,mujeres
2,2016,30,47316,24263,mujeres
3,2016,29,47061,24133,mujeres
4,2016,28,46768,23984,mujeres
...,...,...,...,...,...
1415,2025,49,42339,20255,varones
1416,2025,50,41592,19825,varones
1417,2025,51,40863,19399,varones
1418,2025,45,43952,21226,varones


In [10]:
poblacion_total.columns

Index(['Año', 'Edad', 'Total', 'Total_Parcial', 'genero'], dtype='object')

# 3.3 Creamos nuestra primer tabla: poblacion_total_anio
Esta tabla contiene los datos de la cantidad de población de la Cuidad de Buenos Aires total por año .

In [11]:
poblacion_total_anio = poblacion_total[['Año','Edad','Total_Parcial']]
# Creamos una table pivot para el total de la poblacion por año
poblacion_total_anio = poblacion_total_anio.pivot_table(index='Año',aggfunc='sum')
poblacion_total_anio.drop(['Edad'], axis=1,inplace=True)
# Reseteamos el index del dataframe agrupado para que 'Año' sea una columna regular
poblacion_total_anio.reset_index(inplace=True)
poblacion_total_anio.rename(columns={'Total_Parcial': 'Total'}, inplace=True)
poblacion_total_anio

,Año,Total
0,2016,3059052
1,2017,3063657
2,2018,3067971
3,2019,3071960
4,2020,3075573
5,2021,3078774
6,2022,3081483
7,2023,3083694
8,2024,3085411
9,2025,3086609


# 3.4 Creamos nuestra primer tabla: poblacion_total_anio_genero
Esta tabla contiene los datos de la cantidad de población de la Cuidad de Buenos Aires por año y distinguiendo por género y luego el total.

In [12]:
# Agrupamos por 'Año', 'Edad', y  'genero' y luego sumamos en 'Total_Parcial'
poblacion_total_anio_genero = poblacion_total.groupby(['Año', 'Edad', 'genero']).agg({'Total_Parcial': 'sum'}).reset_index()

# Creamos una tabla Pivot del DataFrame para que 'Año' sea el index, 'Edad' columna, y la suma en 'Total_Parcial' para cada 'genero'  los valores
poblacion_total_anio_genero = poblacion_total_anio_genero.pivot_table(index=['Año', 'genero'], columns='Edad', values='Total_Parcial', aggfunc='sum').reset_index()

# Sumamos por 'Edad'  para obtener  'Total_Parcial'
poblacion_total_anio_genero['Total_Parcial'] = poblacion_total_anio_genero.iloc[:, 2:].sum(axis=1)

# Eliminamos las columnas de Edad 
poblacion_total_anio_genero.drop(columns=range(0, 71), inplace=True)

poblacion_total_anio_genero

Edad,Año,genero,Total_Parcial
0,2016,mujeres,1628557
1,2016,varones,1430495
2,2017,mujeres,1629370
3,2017,varones,1434287
4,2018,mujeres,1630070
5,2018,varones,1437901
6,2019,mujeres,1630643
7,2019,varones,1441317
8,2020,mujeres,1631063
9,2020,varones,1444510


In [13]:
poblacion_total_anio_genero.columns

Index(['Año', 'genero', 'Total_Parcial'], dtype='object', name='Edad')

# 3.5 Creamos una nueva tabla por Rango Etario. 
Vamos a crear una segunda tabla con la información de la cantidad de población por rango etario<br>
Siendo los rangos:
- 0 a 17 años
- 18 a 29 años
- 30 a 49 años
- 50 a 64 años
- mas de 65 años

In [14]:
%run -i funciones.py
# Defino los rangos de edad
rango_edad = {
    '0-17': range(0, 17),
    '18-29': range(18, 29),
    '30-49': range(30, 49),
    '50-65': range(50, 65),
    '66+': [66]
}
# Applicamos la funcion assign_age_group para crear la columna  'Age_range'
poblacion_total['Age_range'] = poblacion_total['Edad'].apply(assign_age_group)

# Agrupamos por 'Age_range', 'Año', y sumamos 'Total_Parcial'
poblacion_total_rango_edad = poblacion_total.groupby(['Age_range', 'Año'])['Total_Parcial'].sum().reset_index()

poblacion_total_rango_edad


,Age_range,Año,Total_Parcial
0,0-17,2016,685901
1,0-17,2017,686612
2,0-17,2018,686201
3,0-17,2019,684787
4,0-17,2020,682620
5,0-17,2021,679982
6,0-17,2022,676974
7,0-17,2023,673208
8,0-17,2024,668873
9,0-17,2025,664194


In [15]:
poblacion_total_rango_edad.columns

Index(['Age_range', 'Año', 'Total_Parcial'], dtype='object')

In [16]:
poblacion_total_anio

,Año,Total
0,2016,3059052
1,2017,3063657
2,2018,3067971
3,2019,3071960
4,2020,3075573
5,2021,3078774
6,2022,3081483
7,2023,3083694
8,2024,3085411
9,2025,3086609


# 3.6 Guardamos estas tablas que serán luego utilizadas en Power Bi para la creación del dashboard

In [18]:
# Ruta donde guardamos el file y el name_file
pobl_total_rango_edad = 'data\\poblacion_total_rango_edad.csv'
pobl_total_anio = 'data\\poblacion_total_anio.csv'
pobl_total_anio_genero =  'data\\poblacion_total_anio_genero.csv'

# Guardamos el dataframe en un archivo .csv
poblacion_total_rango_edad.to_csv(pobl_total_rango_edad, index=False)
poblacion_total_anio.to_csv(pobl_total_anio,index= False)
poblacion_total_anio_genero.to_csv(pobl_total_anio_genero, index= False)

# Mensaje de confirmación de escritura
print(f'El poblacion_total_rango_edad fue guardado en {pobl_total_rango_edad}')
print(f'El poblacion_total_anio fue guardado en {pobl_total_anio}')
print(f'El poblacion_total_anio_genero fue guardado en {pobl_total_anio_genero}')


El poblacion_total_rango_edad fue guardado en data\poblacion_total_rango_edad.csv
El poblacion_total_anio fue guardado en data\poblacion_total_anio.csv
El poblacion_total_anio_genero fue guardado en data\poblacion_total_anio_genero.csv
